In [13]:
import random
from collections import OrderedDict
from bitarray import bitarray

class MemoryInstruction:
    def __init__(self, processId, readWriteFlag, virtualAddress):
        self.processId = processId
        self.virtualAddress = virtualAddress
        self.readWriteFlag = readWriteFlag
        self.virtualPage = virtualAddress >> 12
    def __lt__(self, other):
        return self.processId < other.processId or self.virtualPage < other.virtualPage

class Trace:
    def __init__(self, n, mmu):
        self.queue = []
        self.max_size = n
        self.mmu = mmu
    def add(self, item):
        if len(self.queue) == self.max_size:
            self.queue.pop(0)
        self.queue.append(item)

    def get_promoted_demoted_pages(self):
        sorted_memory_access = sorted(list(set([item for item in self.queue])))
        demoted_pages = {}
        for i in range(len(sorted_memory_access)):
            print(i,"ENTERED")
            super_page_number = sorted_memory_access[i].virtualPage // 4
            processId = sorted_memory_access[i].processId
            if super_page_number in self.mmu.pid_spte_mapping[processId].mapping:
                j = i+1
                count = 1
                while j < len(sorted_memory_access):
                    candidate_super_page_number = sorted_memory_access[j].virtualPage // 4
                    candidate_processId = sorted_memory_access[j].processId
                    if candidate_super_page_number==super_page_number and candidate_processId==processId:
                        count += 1
                    else:
                        if count<=3:
                            if processId in demoted_pages:
                                demoted_pages[processId].append(super_page_number)
                            else:
                                demoted_pages[processId] = [super_page_number]
                        i = i + count
                        break
                    j += 1
        promoted_pages = {}
        for i in range(len(sorted_memory_access)):

            page_number = sorted_memory_access[i].virtualPage
            print(page_number)
            processId = sorted_memory_access[i].processId
            # Checks if the page is the starting base page in a super page and see if the next 3 pages also are accessed, then promote
            if page_number%4==0 and i+3<len(sorted_memory_access) and sorted_memory_access[i+3].processId==processId and sorted_memory_access[i+3].virtualPage==page_number+3 and page_number//4 not in self.mmu.pid_spte_mapping[processId].mapping:
                if processId in promoted_pages:
                    promoted_pages[processId].append(page_number)
                else:
                    promoted_pages[processId] = [page_number]
                # promoted_pages.append([(page_number,processId)])
        print("promoted_pages: mapping from processId to starting base page to be promoted",promoted_pages)
        return demoted_pages,promoted_pages








class RAM:
    def __init__(self):
        # self.pages = bitarray(2 ** 36)
        # self.pages.setall(0)
        # self.empty_pages = 2 ** 36
        self.free_superpage_list = [(0, 2**32-1)]
        self.free_page_list = [(0,2**34-1)]
        self.base_page_process_mapping = {}
        self.super_page_process_mapping = {}
    def evict_page(self, page):
        raise SwapNotImplemented("SwapNotImplemented")
        # self.pages[page] = 0
        # self.empty_pages += 1

    def merge(self,intervals):
        if not intervals:
            return []

        intervals.sort(key=lambda x: x[0])
        merged = [intervals[0]]

        for interval in intervals[1:]:
            if interval[0] <= merged[-1][1]:
                merged[-1] = (merged[-1][0], max(merged[-1][1], interval[1]))
            else:
                merged.append(interval)

        return merged

    def has_empty_pages(self):
        return len(self.free_page_list)==0 and len(self.free_superpage_list)==0

    def allocate_empty_page(self,alloc_requirement_is_super,process_id):
        page = self.allocate_memory(alloc_requirement_is_super,process_id)
        if alloc_requirement_is_super:
            self.super_page_process_mapping[page] = process_id
        else:
            self.base_page_process_mapping[page] = process_id
        return page

    def is_number_in_intervals(number, intervals):
        left = 0
        right = len(intervals) - 1

        while left <= right:
            mid = (left + right) // 2
            interval = intervals[mid]

            if interval[0] <= number <= interval[1]:
                return True
            elif interval[0] > number:
                right = mid - 1
            else:
                left = mid + 1

        return False



    def add_single_pages_to_free_list(self,single_pages):

        self.free_page_list.extend([(p,p) for p in single_pages])
        self.free_page_list = self.merge(self.free_page_list)

        for page in single_pages:
            basepage = (page//4)*4
            count = 0
            for i in range(4):
                if(self.is_number_in_intervals(page+i)):
                    count+=1
            if count == 4:
                self.free_superpage_list.extend((basepage//4,basepage//4))
                self.merge(self.free_superpage_list)



        #     self.free_page_list.append(page,page)
        #
        #     for i in len(self.free_page_list):
        #         page_range = self.free_page_list[i]
        #         if page > page_range[0]



    def map_page_to_process(self,page_num, process_id):
        self.page_table[page_num] = process_id

    def map_superpage_to_process(self,super_page_num,process_id):
        self.super_page_table[super_page_num]  = process_id

    def unmap_page_from_process(self,page_num):
        if page_num in self.page_table:
            del self.page_table[page_num]

    def unmap_superpage_from_process(self,super_page_num):
        if super_page_num in self.super_page_table:
            del self.super_page_table[super_page_num]
    def remove_from_free_list(self,free_list,pages_to_remove):
        updated_free_page_list = []
        # print("AAAAA",self.free_page_list)
        current_start = free_list[0][0]

        for page_range in free_list:
            start, end = page_range

            # Check if any pages to be removed fall within this range
            if start <= pages_to_remove[0] and end >= pages_to_remove[-1]:
                # Add range of pages before the ones to be removed
                if current_start < pages_to_remove[0]:
                    updated_free_page_list.append((current_start, pages_to_remove[0] - 1))

                # Move the start pointer to after the pages to be removed
                current_start = pages_to_remove[-1] + 1

                # If there are more pages after the ones to be removed, add the range to the updated list
                if end > pages_to_remove[-1]:
                    updated_free_page_list.append((current_start, end))

            # If the current range doesn't contain any pages to be removed, just add it to the updated list
            else:
                updated_free_page_list.append((start, end))

        return updated_free_page_list


    def allocate_memory(self,alloc_requirement_is_super,process_id):
        # IF RAM IS NOT FULL
        if alloc_requirement_is_super:
            if len(self.free_superpage_list) > 0:
                # get the last element of the super page list
                superpage = self.free_superpage_list.pop()
                start, end = superpage

                # add the remaining free space to the super page list
                if start + 1 <= end:
                    self.free_superpage_list.append((start + 1, end))
                # remove corresponding single pages from free_page_list
                free_page_start = 4 * (start - 1) + 1
                free_page_end = 4 * (start - 1) + 4
                pages_to_remove = [free_page_start,free_page_start+1,free_page_start+2,free_page_start+3]
                self.super_page_table[start] = process_id
                print("BEFORE",self.free_page_list)
                self.free_page_list = self.remove_from_free_list(self.free_page_list,pages_to_remove)
                return start
            else:
                return None

        # IF RAM IS FULL

        # That means no superpages are available
        #  Swap from a random process physical pages using Page tables

        else:
            if len(self.free_page_list) > 0:
                page = self.free_page_list.pop()
                start, end = page
                if start + 1 <= end:
                    self.free_page_list.append((start + 1, end))

                superpage = start // 4
                # self.free_superpage_list = [sp for sp in self.free_super_pl if sp != superpage]
                print("BEFORE",self.free_page_list)
                self.free_superpage_list = self.remove_from_free_list(self.free_superpage_list,[superpage])
                return start
            else:
                raise MemoryFull("RAM is full")
                # return swapmemory(process_id)



class TLBMiss(Exception):
    pass


class PageFault(Exception):
    pass

class MemoryFull(Exception):
    pass

class SwapNotImplemented(Exception):
    pass

class TLB:
    def __init__(self):
        self.processID = None
        self.mapping = OrderedDict()
    def __str__(self):
        return f"TLB Mapping: {self.mapping}"

    def get_entry(self, virtual_page):
        if virtual_page in self.mapping:
            self.mapping.move_to_end(virtual_page)
            return self.mapping[virtual_page]
        else:
            raise TLBMiss(f"TLB miss: virtual page {virtual_page} has no physical page mapping.")

    def add_entry(self, virtual_page, physical_page):
        self.mapping[virtual_page] = physical_page
        # If the TLB has more than 10 entries, remove the least recently used one
        if len(self.mapping) > 10:
            self.mapping.popitem(last=False)

    def clear_TLB(self):
        self.processID = None
        self.mapping = OrderedDict()

    def remove_entry(self, virtual_page):
        self.mapping.pop(virtual_page)


class STLB:
    def __init__(self):
        self.processID = None
        self.mapping = OrderedDict()
    def __str__(self):
        return f"STLB Mapping: {self.mapping}"
    def get_entry(self, virtual_page):
        if virtual_page in self.mapping:
            self.mapping.move_to_end(virtual_page)
            return self.mapping[virtual_page]
        else:
            raise TLBMiss(f"TLB miss: virtual page {virtual_page} has no physical page mapping.")

    def add_entry(self, virtual_page, physical_page):
        self.mapping[virtual_page] = physical_page
        # If the TLB has more than 10 entries, remove the least recently used one
        if len(self.mapping) > 10:
            self.mapping.popitem(last=False)

    def remove_entry(self, virtual_page):
        if virtual_page in self.mapping:
            self.mapping.pop(virtual_page)
        else:
            raise Exception(f"Super page {virtual_page} doesnt exist in STLB")

    def clear_STLB(self):
        self.processID = None
        self.mapping = OrderedDict()


class PTE:
    def __init__(self):
        self.processID = None
        self.mapping = OrderedDict()

    def __str__(self):
        return f"PTE Mapping: {self.mapping}"

    def get_physical_page(self, virtual_page):
        if virtual_page not in self.mapping:
            # If the virtual page is not in the mapping, generate a page fault
            raise PageFault(f"Page fault: virtual page {virtual_page} does not exist.")
        elif self.mapping[virtual_page] is None:
            # If the physical page is None, generate a page fault
            raise PageFault(f"Page fault: virtual page {virtual_page} has no physical page mapping.")
        else:
            # If the mapping exists, move the virtual page to the end so that the first element denotes the LRU entry
            self.mapping.move_to_end(virtual_page)
            physical_page = self.mapping[virtual_page]
            return physical_page

    def evict_lru_page(self):
        # Remove the least recently used virtual page and return its mapping
        virtual_page, physical_page = self.mapping.popitem(last=False)
        return (virtual_page, physical_page)

    def add_mapping(self, virtual_page, physical_page):
        # Add a virtual-to-physical page mapping to the mapping dictionary
        self.mapping[virtual_page] = physical_page

    def remove_entry(self, virtual_page):
        self.mapping.pop(virtual_page)


class SPTE:
    def __init__(self):
        self.processID = None
        self.mapping = OrderedDict()
    def __str__(self):
        return f"SPTE Mapping: {self.mapping}"
    def get_physical_page(self, virtual_page):
        if virtual_page not in self.mapping:
            # If the virtual page is not in the mapping, generate a page fault
            raise PageFault(f"Page fault: virtual page {virtual_page} does not exist.")
        elif self.mapping[virtual_page] is None:
            # If the physical page is None, generate a page fault
            raise PageFault(f"Page fault: virtual page {virtual_page} has no physical page mapping.")
        else:
            # If the mapping exists, move the virtual page to the end so that the first element denotes the LRU entry
            self.mapping.move_to_end(virtual_page)
            physical_page = self.mapping[virtual_page]
            return physical_page

    def evict_lru_page(self):
        # Remove the least recently used virtual page and return its mapping
        virtual_page, physical_page = self.mapping.popitem(last=False)
        return (virtual_page, physical_page)

    def add_mapping(self, virtual_page, physical_page):
        # Add a virtual-to-physical page mapping to the mapping dictionary
        self.mapping[virtual_page] = physical_page

    def remove_entry(self, virtual_page):
        self.mapping.pop(virtual_page)


class MMU:
    def __init__(self):
        self.pid_spte_mapping = {}
        self.pid_pte_mapping = {}
        self.page_metadata = {}
        self.tlb = TLB()
        self.stlb = STLB()
        self.RAM = RAM()  # 2^36 pages in RAM, each page is 2^12 bytes -- Total 48 bits
        self.tlbMisses = 0
        # maintain same variable for tlb mapping and pid_pte_mapping

    def initialize_page_table_for_process(self, processID):
        # Initialize a new PTE table for the given processID and add it to the mapping
        self.pid_pte_mapping[processID] = PTE()
        self.pid_pte_mapping[processID].processID = processID

    def initialize_superpage_table_for_process(self, processID):
        self.pid_spte_mapping[processID] = SPTE()
        self.pid_spte_mapping[processID].processID = processID

    def clear_TLB(self):
        # Clear the TLB by reinitializing it
        self.tlb.clear_TLB()

    def get_physical_address(self, ins: MemoryInstruction):
        processId = ins.processId
        virtual_address = ins.virtualAddress
        if processId not in self.pid_pte_mapping:
            self.initialize_page_table_for_process(processId)
            self.initialize_superpage_table_for_process(processId)
        virtual_page = virtual_address >> 12
        # Check if the physical address is in the TLB
        try:
            physical_page = self.tlb.get_entry(virtual_page)
        except TLBMiss as e:
            self.tlbMisses += 1
            try:
                # Look up the PTE entry for the processID
                pte_entry = self.pid_pte_mapping[processId]

                physical_page = pte_entry.get_physical_page(virtual_page)
            except PageFault as e:
                physical_page = self.get_free_page(processId)
                self.add_pte_mapping(processId, virtual_page, physical_page)
            # Update the TLB
            self.tlb.add_entry(virtual_page, physical_page)

        offset = virtual_address & 0xFFF
        physical_address = (physical_page << 12) | offset
        return physical_address

    def get_free_page(self,processId):
        # Get a free physical page from the unused_pages set, or evict an LRU page
        if not self.RAM.has_empty_pages():
            physical_page = self.RAM.allocate_empty_page(False,processId)
        else:
            # pick a process with maximum pages in PTE entry and evict it to harddisk
            processID, virtual_page = max(self.pid_pte_mapping.items(), key=lambda x: len(x[1].mapping))
            virtual_page, physical_page = self.pid_pte_mapping[processID].evict_lru_page()
        return physical_page

    def add_pte_mapping(self, processID, virtual_page, physical_page):
        # Add a virtual-to-physical page mapping to the PTE entry for the given processID
        pte_entry = self.pid_pte_mapping[processID]
        pte_entry.mapping[virtual_page] = physical_page

    def add_spte_mapping(self, processID, virtual_page, physical_page):
        # Add a virtual-to-physical page mapping to the PTE entry for the given processID
        pte_entry = self.pid_spte_mapping[processID]
        pte_entry.mapping[virtual_page] = physical_page

    def remove_spte_mapping(self, processID):
        # Add a virtual-to-physical page mapping to the PTE entry for the given processID
        # pte_entry = self.pid_spte_mapping[processID]
        # pte_entry.mapping[virtual_page] = physical_page
        self.pid_spte_mapping.pop(processID)


def get_superpage_pid():
    return {1: [5]}


def get_physical_super_page(mmu: MMU, v_spg, pid):
    return mmu.pid_spte_mapping[pid].get_entry(v_spg)


def get_basepgs_vpgs(v_spg):
    return [4*v_spg, 4*v_spg+1, 4*v_spg+2, 4*v_spg+3]


def get_basepgs_ppgs(p_spg):
    return [4*p_spg, 4*p_spg+1, 4*p_spg+2, 4*p_spg+3]


def remove_stlb_entries(mmu, vspg):
    try:
        mmu.stlb.remove_entry(vspg)
    except Exception as e:
        print(f"Cannot demote {vspg} as it doesnt exist in STLB")


def remove_spte_entries(mmu, pid):
    try:
        #mmu.pid_spte.mapping[vspg] = None
        mmu.remove_spte_mapping(pid)
    except Exception as e:
        print(f"Cannot remove processid : {pid} from SPTE as it doesnt exist")


def add_tlb_entries(mmu, v_basepgs, p_basepgs):
    for i in range(len(v_basepgs)):
        mmu.tlb.add_entry(v_basepgs[i], p_basepgs[i])


def add_pte_entries(mmu, v_basepgs, p_basepgs, pid):
    for i in range(len(v_basepgs)):
        mmu.add_pte_mapping(pid, v_basepgs[i], p_basepgs[i])


def demotion(mmu, data, pid):
    remove_stlb_entries(mmu, data['super_virtual_pg'])
    remove_spte_entries(mmu, data['pid'])
    add_pte_entries(mmu, data['virtual_pgs'], data['physical_pgs'], data['pid'])


def perform_demotion(mmu, demotion_pid_superpagesmap):
    for pid in demotion_pid_superpagesmap:
        super_pages = demotion_pid_superpagesmap[pid]
        # data= vpage , super_page_pid[data] = pid
        p_spg = get_physical_super_page(mmu, v_spg, pid)
        v_base_pgs = get_basepgs_vpgs(v_spg)
        p_base_pgs = get_basepgs_ppgs(p_spg)
        demotion(mmu, {"virtual_pgs": v_base_pgs, "physical_pgs": p_base_pgs, "pid": super_page_pid[v_spg], "super_virtual_pg": v_spg, "super_physical_pg": p_spg})


def get_physical_base_pages(mmu, pid, vbase_pgs):
    list = []
    pte = mmu.pid_pte_mapping[pid]
    for vpg in vbase_pgs:
        base_ppg = pte.get_physical_page(vpg)
        list.append(base_ppg)
    return list


def remove_pte_entries(mmu, pid, vbase_page):
    pte = mmu.pid_pte_mapping[pid]
    physical_page = pte[vbase_page]
    pte.remove(vbase_page)
    return physical_page





def remove_tlb_entries(mmu, vbase_pgs):
    for vpg in vbase_pgs:
        mmu.tlb.remove_entry(vpg)


def get_physical_super_page_addr(pbase_pgs):
    return (pbase_pgs[0]) // 4


def get_virtual_super_page_addr(base_pg):
    return base_pg // 4


def promotion(mmu: MMU, base_page_pid):
    ram = mmu.RAM
    for pid in mmu.pid_pte_mapping:
        print("Before promotion PTE :",pid,mmu.pid_pte_mapping[pid])
    for pid in mmu.pid_spte_mapping:
        print("Before promotion SPTE :",pid,mmu.pid_spte_mapping[pid])

    print("Before promotion tlb,Stlb :",mmu.tlb ,mmu.stlb)


    for pid in base_page_pid:
        # pbase_pgs = get_physical_base_pages(mmu, pid, base_page_pid[pid])
        for base_page in base_page_pid[pid]:
            for i in range(4):
                removed_physical_page = remove_pte_entries(mmu, pid, base_page+i)
                ram.add_single_pages_to_free_list([removed_physical_page])
                remove_tlb_entries(mmu, base_page + i)
            s_virtual_page = get_virtual_super_page_addr(base_page)
            s_physical_page = ram.allocate_empty_page(True,pid)
        # remove_pte_entries(mmu, pid, base_page_pid[pid])

        mmu.stlb.add_entry(s_virtual_page, s_physical_page)

        if pid not in mmu.pid_spte_mapping:
            mmu.initialize_superpage_table_for_process(pid)
        spte = mmu.pid_spte_mapping[pid]
        spte.add_mapping(s_virtual_page, s_physical_page)



    for pid in mmu.pid_pte_mapping:
        print("AFTER promotion PTE :",pid,mmu.pid_pte_mapping[pid])
    for pid in mmu.pid_spte_mapping:
        print("AFTER promotion SPTE :",pid,mmu.pid_spte_mapping[pid])

    print("AFTER promotion tlb,Stlb :",mmu.tlb ,mmu.stlb)

def get_basepage_pid():
    return {5: [1, 2, 3, 4]}



# def main():
#     # import pdb
#     # pdb.set_trace()
#     processes = [1, 2, 3, 4, 5]
#     virtual_address_size = 2 ** 48
#     mmu = MMU()
#     previousProcessId = 0
#
#     processId = 5
#     virtual_address = random.randint(0, virtual_address_size - 1) #get new addresses (set logic)
#     # virtual_address = 8192
#     # if previousProcessId is not processId:
#     #     mmu.clear_TLB()
#     # physical_address = mmu.get_physical_address(virtual_address, processId)
#     #         print(f"{processId},{virtual_address},{physical_address}")
#     #previousProcessId = processId
#     #mmu.initialize_page_table_for_process(1)
#
#     # mmu.initialize_superpage_table_for_process(processId)
#     # mmu.add_pte_mapping(processId, 1, 9)
#     # mmu.add_pte_mapping(processId, 2, 10)
#     # mmu.add_pte_mapping(processId, 3, 11)
#     # mmu.add_pte_mapping(processId, 4, 12)
#     # mmu.tlb.add_entry(1, 9)
#     # mmu.tlb.add_entry(2, 10)
#     # mmu.tlb.add_entry(3, 11)
#     # mmu.tlb.add_entry(4, 12)
#
#     # print(physical_address)
#     print("TLB_Misses", mmu.tlbMisses)
#     # import pdb
#     # pdb.set_trace()
#     # super_page_pid = get_superpage_pid()
#     # perform_demotion(mmu, super_page_pid)
#
#     base_page_pid = get_basepage_pid()
#     promotion(mmu, base_page_pid)
#     #print(mmu.stlb.get_entry(1))
#     pte = mmu.pid_pte_mapping[processId]
#     print(pte.get_physical_page(1))
# # add dirty bit
#
if __name__ == "__main__":
    print("hello")




hello


In [12]:
mmu = MMU()
n = 10000

trace = Trace(n,mmu)

ins1 = MemoryInstruction(1,"W",0<<12)
ins2 = MemoryInstruction(1,"W",1<<12)
ins3 = MemoryInstruction(1,"W",2<<12)
ins4 = MemoryInstruction(1,"W",3<<12)

print(mmu.get_physical_address(ins1))
print(mmu.get_physical_address(ins2))
print(mmu.get_physical_address(ins3))
print(mmu.get_physical_address(ins4))
# mmu.get_physical_address(ins3)
# mmu.get_physical_address(ins4)
trace.add(ins1)
trace.add(ins2)
trace.add(ins3)
trace.add(ins4)
print(trace.get_promoted_demoted_pages())
demoted_pages,promoted_pages = trace.get_promoted_demoted_pages()
print(promoted_pages)
promotion(mmu,base_page_pid = promoted_pages)



BEFORE [(1, 17179869183)]
0
BEFORE [(2, 17179869183)]
4096
BEFORE [(3, 17179869183)]
8192
BEFORE [(4, 17179869183)]
12288
0 ENTERED
1 ENTERED
2 ENTERED
3 ENTERED
0
1
2
3
promoted_pages: mapping from processId to starting base page to be promoted {1: [0]}
({}, {1: [0]})
0 ENTERED
1 ENTERED
2 ENTERED
3 ENTERED
0
1
2
3
promoted_pages: mapping from processId to starting base page to be promoted {1: [0]}
{1: [0]}
Before promotion PTE : 1 PTE Mapping: OrderedDict([(0, 0), (1, 1), (2, 2), (3, 3)])
Before promotion SPTE : 1 SPTE Mapping: OrderedDict()
Before promotion tlb,Stlb : TLB Mapping: OrderedDict([(0, 0), (1, 1), (2, 2), (3, 3)]) STLB Mapping: OrderedDict()


NameError: name 'vpg' is not defined

NameError: name 'MemoryInstruction' is not defined

In [5]:
def main():
    processes = [1, 2, 3, 4, 5]
    virtual_address_size = 2**48
    mmu = MMU()
    previousProcessId = 0
    for i in range(100):
        processId = random.choice(processes)
        previous_virtual_address = 0
        for _ in range(1000):
            virtual_address = random.randint(0, virtual_address_size - 1)
            if previousProcessId is not processId:
                mmu.clear_TLB()
            physical_address = mmu.get_physical_address(virtual_address, processId)
            #         print(f"{processId},{virtual_address},{physical_address}")
            previousProcessId = processId
    print("TLB_Misses",mmu.tlbMisses)
if __name__ == "__main__":
    main()



TypeError: MMU.get_physical_address() takes 2 positional arguments but 3 were given